# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,20.91,81,20,3.60,US,1707887472
1,1,diu,20.7141,70.9822,26.82,21,2,4.42,IN,1707887473
2,2,marka,1.7159,44.7717,27.03,80,32,6.26,SO,1707887474
3,3,muroto-misakicho,33.2833,134.1500,15.89,68,100,1.60,JP,1707887476
4,4,la paz,-16.5000,-68.1500,9.21,83,99,1.34,BO,1707887477


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
x = city_data_df['Lat']
y = city_data_df['Lng']
size = city_data_df['Humidity'].tolist()

map_plot = city_data_df.hvplot.points(y,x,size =size, geo = True, tiles = 'OSM', frame_height = 600, frame_width = 800)
# Display the map
# YOUR CODE HERE
map_plot

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
min_temp = 21
max_temp = 27
max_wind_speed = 4.5
cloudiness = 0
# Drop any rows with null values
# YOUR CODE HERE
ideal_cities_df = city_data_df[(city_data_df['Max Temp'] > min_temp) & 
                              (city_data_df['Max Temp'] < max_temp) &
                              (city_data_df['Wind Speed'] < max_wind_speed) &
                              (city_data_df['Cloudiness'] == cloudiness)]

ideal_cities_df = ideal_cities_df.dropna()
# Display sample data
# YOUR CODE HERE
ideal_cities_df.sample(n=10, replace = True)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
317,317,brisas de zicatela,15.8369,-97.0419,25.00,81,0,1.84,MX,1707887459
8,8,taunsa,30.7048,70.6505,21.17,15,0,0.33,PK,1707887303
408,408,pisco,-13.7000,-76.2167,23.03,83,0,4.12,PE,1707887914
335,335,cockburn town,21.4612,-71.1419,25.51,75,0,4.24,TC,1707887838
142,142,east london,-33.0153,27.9116,21.52,78,0,4.12,ZA,1707887627
408,408,pisco,-13.7000,-76.2167,23.03,83,0,4.12,PE,1707887914
335,335,cockburn town,21.4612,-71.1419,25.51,75,0,4.24,TC,1707887838
142,142,east london,-33.0153,27.9116,21.52,78,0,4.12,ZA,1707887627
53,53,papatowai,-46.5619,169.4708,21.38,48,0,1.18,NZ,1707887532
335,335,cockburn town,21.4612,-71.1419,25.51,75,0,4.24,TC,1707887838


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy(deep = True)
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""
# Display sample data
# YOUR CODE HERE
hotel_df.sample(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
84,punta arenas,CL,-53.1500,-70.9167,76,
332,porto novo,BJ,6.4965,2.6036,94,
261,fergus falls,US,46.2830,-96.0776,93,
193,fale old settlement,TK,-9.3852,-171.2468,83,
5,puerto leguizamo,CO,-0.1934,-74.7819,98,
532,queenstown,ZA,-31.8976,26.8753,80,
170,xining,CN,36.6167,101.7667,47,
495,berdigestyakh,RU,62.0989,126.6992,88,
429,nikolayevsk-on-amure,RU,53.1406,140.7300,72,
251,cheat lake,US,39.6720,-79.8534,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',
          'apiKey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = row['Lat']
    lon = row['Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
diu - nearest hotel: Hotel Ankur
marka - nearest hotel: Salsabiil Hotel
muroto-misakicho - nearest hotel: 民宿うらしま
la paz - nearest hotel: Alojamiento Tokio
puerto leguizamo - nearest hotel: Hotel Girasoles
port-aux-francais - nearest hotel: Keravel
grytviken - nearest hotel: No hotel found
taunsa - nearest hotel: No hotel found
whitehorse - nearest hotel: Town & Mountain Hotel
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
iqaluit - nearest hotel: Frobisher Inn
blackmans bay - nearest hotel: Villa Howden
mili - nearest hotel: No hotel found
satkania - nearest hotel: Mayer Doya Hotel & Jhal Bitan
bethel - nearest hotel: Hampton Inn Danbury
atar - nearest hotel: فندق سكليل
kingscliff - nearest hotel: Kingscliff Beach Hotel
zheshart - nearest hotel: No hotel found
bonthe - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
port m

,City,Country,Lat,Lng,Humidity,Hotel Name
0,wailua homesteads,US,22.0669,-159.3780,81,Hilton Garden Inn Kauai Wailua Bay
1,diu,IN,20.7141,70.9822,21,Hotel Ankur
2,marka,SO,1.7159,44.7717,80,Salsabiil Hotel
3,muroto-misakicho,JP,33.2833,134.1500,68,民宿うらしま
4,la paz,BO,-16.5000,-68.1500,83,Alojamiento Tokio
...,...,...,...,...,...,...
564,muynoq,UZ,43.7683,59.0214,51,Lala
565,fisterra,ES,42.9049,-9.2629,98,Hotel Temático do Banco Azul
566,leh,IN,34.1667,77.5833,53,Kang Lha Chen
567,ajaokuta,NG,7.5623,6.6550,69,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_map_plot = hotel_df.hvplot.points('Lng','Lat', 
                                        size = size, 
                                        geo = True, 
                                        tiles='OSM', 
                                        frame_height = 600, 
                                        frame_width = 800, 
                                        hover_cols = ['City', 'Country', 'Hotel Name', 'Humidity'])
# Display the map
# YOUR CODE HERE
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)